In [8]:
import drjit as dr
import mitsuba as mi
mi.set_variant('cuda_ad_rgb', 'llvm_ad_rgb')


In [118]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from matplotlib import pyplot as plt

In [119]:
#  s / (a + s)
ink_albedo = np.array([
    [0.05, 0.7, 0.98],  # Cyan
    [0.98, 0.1, 0.9],  # Magenta
    [0.997, 0.995, 0.15],  # Yellow
    [0.35, 0.35, 0.35],  # KEY: Black
    [0.9991, 0.9997, 0.999],   # White
    [1.0, 1.0, 1.0] #Transparent
    ])
# ink_albedo = torch.tensor(ink_albedo, dtype=torch.float32)

# a + s
ink_sigma_t = np.array([
        [9.0, 4.5, 7.5],  # Cyan
        [2.5, 3.0, 10.0],  # Magenta
        [2.25, 3.75, 19.0],  # Yellow
        [5.0, 5.5, 6.5],  # KEY: Black
        [6.0, 9.0, 24.0],   # White
        [1e-4, 1e-4, 1e-4]] #Transparent
        ) /20
# ink_sigma_t = torch.tensor(ink_sigma_t, dtype=torch.float32)

has_transparent = False
weight = np.random.rand(6)
if not has_transparent:
    weight[5] = 0.0
weight = weight / weight.sum()
weight = torch.tensor(weight, dtype=torch.float32)

In [120]:
ink_scattering = ink_albedo * ink_sigma_t
ink_absorption = ink_sigma_t - ink_scattering

ink_absorption = torch.tensor(ink_absorption, dtype=torch.float32)
ink_scattering = torch.tensor(ink_scattering, dtype=torch.float32)

## Comparison scene setup ##
- center [5. 0. 0.]
- scale [290. 240. 240.]

In [151]:
from mitsuba import ScalarTransform4f as T

sensor_to_world = T.look_at(target=[0, 0, 0], origin=[0, 0, 5], up=[0, 1, 0])
print(sensor_to_world)
# sensor_to_world = mi.TensorXf([1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,-1.8,0.0,0.0,0.0,1.0])
sensor_thickness_experiment = mi.load_dict({
    'type': 'perspective',
    'fov': 39,
    'to_world': sensor_to_world,
    'film': {
        'type': 'hdrfilm',
        'width': 64, 'height': 64,
        'filter': {'type': 'tent'}
    }
})

[[-1, 0, 0, 0],
 [0, 1, 0, 0],
 [0, 0, -1, 5],
 [0, 0, 0, 1]]


In [154]:
from mitsuba.scalar_rgb import Transform4f as T

def load_scene_diff_cube(bbox_center, bbox_scale):

    translate_3 = - bbox_scale[2] + 20
    scene = mi.load_dict({
        'type': 'scene',
        'integrator': {'type': 'volpathmis'},
        'red_cude': {
            'type': 'obj',
            'filename': 'set1/meshes/Cube.obj',
            # 'bsdf':{'type': 'diffuse',
            # 'reflectance': {
            #     'type': 'rgb',
            #     'value': [0.2, 0.25, 0.7]
            # }}
            'bsdf': {'type': 'null'},
            'interior': {
                'type': 'homogeneous',
                'albedo': {
                    # 'type': 'rgb',
                    # 'value': [1, 0.0, 0.0],
                    'type': 'gridvolume',
                    'filename': 'set1/meta/albedo.vol',
                    'filter_type': 'trilinear',
                    'to_world': T.rotate([1, 0, 0], -90).translate(list(bbox_center)).scale(list(bbox_scale)),

                },
                'sigma_t': {
                    # 'type': 'rgb',
                    # 'value': [1, 0.0, 0.0],
                    # 'type': 'gridvolume',
                    'filename': 'set1/meta/sigma_t.vol',
                    'filter_type': 'trilinear',
                    'to_world': T.rotate([1, 0, 0], -90).translate(list(bbox_center)).scale(list(bbox_scale)),

                },
                'scale': 20
            }
        },
        'emitter': {'type': 'constant'}
    })

    return scene



In [155]:
# w = torch.tensor([0.0000, 0.5107, 0.4893, 0.0000, 0.0000, 0.0000])
# sigma_t = (w @ (ink_absorption+ ink_scattering))
# albedo = (w @ ink_scattering / sigma_t)
# sigma_t = sigma_t.detach().numpy()
# albedo = albedo.detach().numpy()

bbox_center = np.array([0.0, 0.0, 0.0])
bbox_scale = np.array([290.0, 240.0, 240.0])


scene = load_scene_diff_cube(bbox_center, bbox_scale)
# key_albedo = 'red_cude.interior_medium.albedo.value.value'
# key_sigma_t = 'red_cude.interior_medium.sigma_t.value.value'
# params = mi.traverse(scene)
# params[key_albedo] = albedo
# params[key_sigma_t] = sigma_t
# params.update()
img = mi.render(scene,sensor=sensor_thickness_experiment, spp=4)
mi.util.convert_to_bitmap(img)


Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [64, 64],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 12 KiB of image data ]
]